# Modeling

## Building the Model

In [2]:
import numpy as np
import pandas as pd

X_train = np.load('train:val:test/X_train.npy')
y_train = np.load('train:val:test/y_train.npy')
X_val = np.load('train:val:test/X_val.npy')
y_val = np.load('train:val:test/y_val.npy')
X_test = np.load('train:val:test/X_test.npy')

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)

X_test = X_test.astype(np.float32)

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LSTM, Masking
from tensorflow.keras.models import Sequential

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    Masking(mask_value=0.0),
    LSTM(16),
    Dense(8),
    Dense(1, 'linear')
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 4, 31)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16)             │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,217 (12.57 KB)

 Trainable params: 3,217 (12.57 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
)

model.fit(X_train_scaled[:5], y_train[:5], epochs=15)
# validation_data=(X_val_scaled, y_val)

Epoch 1/15


In [ ]:
y_pred = model.predict(X_val_scaled)
y_pred = y_pred.flatten()
y_pred = np.round(y_pred)
d = {"Actual Home Runs": y_val, "Predicted Home Runs": y_pred}
comparison = pd.DataFrame(data=d)
comparison

In [ ]:
tigers = ['Javier Báez', 'Kerry Carpenter', 'Dillon Dingler', 'Riley Greene', 'Colt Keith',
         'Justyn Henry Malloy', 'Zach Mckinstry', 'Parker Meadows', 'Wenceel Pérez',
         'Jake Rogers', 'Spencer Torkelson', 'Gleyber Torres', 'Matt Vierling']

tigers_pred = model.predict(X_test_scaled)
tigers_pred = tigers_pred.flatten()
tigers_pred = np.round(tigers_pred)

dtown = {"Detroit Tiger": tigers, "Predicted Home Runs": tigers_pred}
projections = pd.DataFrame(data=dtown)
projections